In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, make_scorer
import seaborn as sns

In [4]:
data_path = "/home/dhauryk/!DUNAI/train_ver2.csv"
df = pd.read_csv(data_path)

df = df.sort_values(["ncodpers", "fecha_dato"])

/tmp/ipykernel_17698/7997590.py:2: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [5]:
# Define target columns (product subscriptions)
target_cols = [col for col in df.columns if col.startswith("ind_")]
target_cols.append("ncodpers") 

# Feature columns (customer attributes)
feature_cols = ["ind_empleado", "pais_residencia", "sexo", "age", "ind_nuevo", "antiguedad", "nomprov", "segmento"]

dtype_list = {"ncodpers": "int64"}
for col in df.columns:
    if col.startswith("ind_"):
        dtype_list.setdefault(col, "float16'")

In [6]:
# Data preprocessing function
def preprocess_data(df, feature_cols, target_cols, dtype_list):
    train = df
    
    # Preprocessing logic here
    for col in feature_cols:
        if col == 'age':
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df.loc[df.age < 18, "age"] = df.loc[(df.age >= 18) & (df.age <= 30), "age"].mean(skipna=True)
            df['age'].fillna(df['age'].mean(), inplace=True)
            df['age'] = df['age'].astype(int)
        
        elif col == 'ind_nuevo':
            df.loc[df[col].isnull(), col] = 1
        
        elif col == 'antiguedad':
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df.loc[df[col].isnull(), col] = df[col].min()
            df.loc[df[col] < 0, col] = 0
        
        elif col == 'nomprov':
            df[col].fillna('Unknown', inplace=True)
        
        elif col == 'segmento':
            df[col] = df[col].apply(lambda x: str(x).split('-')[0])
            df.loc[df[col].isnull(), col] = 'Unknown'
        
        else:
            print(col)
            df[col].fillna(-999, inplace=True)
    
    # Label encoding for categorical columns
    for col in feature_cols:
        if df[col].dtype == 'object':
            le = LabelEncoder()
            le.fit(list(df[col].values))
            df[col] = le.transform(df[col].values)

    for col in df.columns:
        df[col].fillna(-999, inplace=True)
    
    return df


In [7]:
data = preprocess_data(df, feature_cols, target_cols, dtype_list)
# Separate features (X) and target (y) from the training data
X = data[feature_cols]
y = data[target_cols[1:]]  # excluding 'ncodpers'
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

ind_empleado


/tmp/ipykernel_17698/2847176983.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(-999, inplace=True)


pais_residencia
sexo


/tmp/ipykernel_17698/2847176983.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(df['age'].mean(), inplace=True)
/tmp/ipykernel_17698/2847176983.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [ ]:
# Define the custom scorer using make_scorer
def map7_scorer(estimator, X, y):
    y_pred = estimator.predict_proba(X)
    top_k_preds = np.argsort(y_pred, axis=1)[:, -7:]  # Get top 7 predictions
    actuals = [set(np.where(row == 1)[0]) for row in y]  # Convert y into sets of actual labels
    
    return map7(top_k_preds, actuals)

# Create the custom scorer
map7_scorer_obj = make_scorer(map7_scorer, greater_is_better=True)

: 

In [ ]:
# Initialize RandomForest model
clf = RandomForestClassifier(n_estimators=1, max_depth=10, n_jobs=10, verbose=True, random_state=42)

# Cross-validation: We will use StratifiedKFold for multi-label classification
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation score for RandomForestClassifier
cross_val_score_rf = cross_val_score(clf, x_train, y_train, cv=cv, scoring=map7_scorer_obj)

# Training the model
clf.fit(x_train, y_train, v)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


In [ ]:
# Plotting feature importance
feature_importance = clf.feature_importances_
sns.barplot(x=feature_cols, y=feature_importance)
plt.title("Feature Importance")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Prediction and MAP7 calculation
k = 7
y_pred_prob = clf.predict_proba(x_test)
top_k_preds = np.argsort(y_pred, axis=1)[:, -k:]

In [ ]:
def map7(preds, actuals, k=7):
    avg_precisions = []  # Сюда собираем средние точности каждой последовательности
    for pred, actual in zip(preds, actuals):
        match = 0  # Счетчик совпадений
        sum_precisions = 0  # Сумма точности в каждом ранге
        for i, p in enumerate(pred[:k]):  # Берем топ 7
            if p in actual:  # Проверяем есть ли прогноз в актуальном списке
                match += 1
                sum_precisions += match / (i + 1)  # Точность вычисления на i-м ранге
        avg_precisions.append(sum_precisions / min(k, len(actual)))  # Считаем среднюю точность

    return np.mean(avg_precisions)  # Возвращаем среднее значения всех средних значений точности



In [ ]:
# Calculate MAP7 on the validation set
actuals = []  # This should be your test set actual labels
for idx, row in enumerate(test_labels):  # Modify this to fit your actual label format
    actual = set(np.where(row == 1)[0])  # Extract the indices of true labels
    actuals.append(actual)
    
map7_score = map7(top_k_preds, actuals, k)
print(f'MAP@7 Score: {map7_score:.4f}')

In [ ]:




# Define target columns and dtypes
targetcols = ['ncodpers', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

# Data preprocessing function
def preprocess_data(train_file, test_file, feature_cols, targetcols, dtype_list):
    train = pd.read_csv(train_file, usecols=feature_cols + targetcols, dtype=dtype_list)
    test = pd.read_csv(test_file, usecols=feature_cols, dtype=dtype_list)
    
    # Preprocessing logic here
    for col in feature_cols:
        if col == 'age':
            train[col] = pd.to_numeric(train[col], errors='coerce')
            test[col] = pd.to_numeric(test[col], errors='coerce')
            train.loc[train.age < 18, "age"] = train.loc[(train.age >= 18) & (train.age <= 30), "age"].mean(skipna=True)
            test.loc[test.age > 100, "age"] = test.loc[(test.age >= 30) & (test.age <= 100), "age"].mean(skipna=True)
            train['age'].fillna(train['age'].mean(), inplace=True)
            test['age'].fillna(test['age'].mean(), inplace=True)
            train['age'] = train['age'].astype(int)
            test['age'] = test['age'].astype(int)
        
        elif col == 'ind_nuevo':
            train.loc[train[col].isnull(), col] = 1
            test.loc[test[col].isnull(), col] = 1
        
        elif col == 'antiguedad':
            train[col] = pd.to_numeric(train[col], errors='coerce')
            test[col] = pd.to_numeric(test[col], errors='coerce')
            train.loc[train[col].isnull(), col] = train[col].min()
            test.loc[test[col].isnull(), col] = test[col].min()
            train.loc[train[col] < 0, col] = 0
            test.loc[test[col] < 0, col] = 0
        
        elif col == 'nomprov':
            train[col].fillna('Unknown', inplace=True)
            test[col].fillna('Unknown', inplace=True)
        
        elif col == 'segmento':
            train[col] = train[col].apply(lambda x: str(x).split('-')[0])
            test[col] = test[col].apply(lambda x: str(x).split('-')[0])
            train.loc[train[col].isnull(), col] = 'Unknown'
            test.loc[test[col].isnull(), col] = 'Unknown'
        
        else:
            train[col].fillna(-999, inplace=True)
            test[col].fillna(-999, inplace=True)
    
    # Label encoding for categorical columns
    for col in feature_cols:
        if train[col].dtype == 'object':
            le = LabelEncoder()
            le.fit(list(train[col].values) + list(test[col].values))
            train[col] = le.transform(train[col].values)
            test[col] = le.transform(test[col].values)
    
    return train, test

# Load and preprocess the data
train_file = '../input/train_ver2.csv'
test_file = '../input/test_ver2.csv'
feature_cols = ["ind_empleado","pais_residencia","sexo", "age", "ind_nuevo", "antiguedad", "nomprov", "segmento"]
dtype_list = {'ind_cco_fin_ult1': 'float16', 'ind_deme_fin_ult1': 'float16', 'ind_aval_fin_ult1': 'float16',
              'ind_valo_fin_ult1': 'float16', 'ind_reca_fin_ult1': 'float16', 'ind_ctju_fin_ult1': 'float16',
              'ind_cder_fin_ult1': 'float16', 'ind_plan_fin_ult1': 'float16', 'ind_fond_fin_ult1': 'float16',
              'ind_hip_fin_ult1': 'float16', 'ind_pres_fin_ult1': 'float16', 'ind_nomina_ult1': 'float16',
              'ind_cno_fin_ult1': 'float16', 'ncodpers': 'int64', 'ind_ctpp_fin_ult1': 'float16',
              'ind_ahor_fin_ult1': 'float16', 'ind_dela_fin_ult1': 'float16', 'ind_ecue_fin_ult1': 'float16',
              'ind_nom_pens_ult1': 'float16', 'ind_recibo_ult1': 'float16', 'ind_deco_fin_ult1': 'float16',
              'ind_tjcr_fin_ult1': 'float16', 'ind_ctop_fin_ult1': 'float16', 'ind_viv_fin_ult1': 'float16',
              'ind_ctma_fin_ult1': 'float16'}

train, test = preprocess_data(train_file, test_file, feature_cols, targetcols, dtype_list)

# Separate features (X) and target (y) from the training data
X = train[feature_cols]
y = train[targetcols[1:]]  # excluding 'ncodpers'

# Split data into train and validation sets (80-20 split)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomForest model
clf = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1, random_state=42)

# Cross-validation: We will use StratifiedKFold for multi-label classification
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation score for RandomForestClassifier
cross_val_score_rf = cross_val_score(clf, x_train, y_train, cv=cv, scoring='average_precision_score')

# Training the model
clf.fit(x_train, y_train)

# Plotting feature importance
feature_importance = clf.feature_importances_
sns.barplot(x=feature_cols, y=feature_importance)
plt.title("Feature Importance")
plt.xticks(rotation=90)
plt.show()

# Prediction and MAP7 calculation
y_pred_prob = clf.predict_proba(x_test)

# Compute MAP7: Mean Average Precision at 7
def map_at_k(y_true, y_pred_prob, k=7):
    avg_precision = []
    for i in range(y_true.shape[0]):
        true_labels = np.where(y_true[i] == 1)[0]  # ground truth labels
        pred_labels = np.argsort(y_pred_prob[i])[::-1][:k]  # top k predicted labels
        avg_precision.append(average_precision_score([true_labels], [pred_labels]))  # AP score per instance
    return np.mean(avg_precision)

# Calculate MAP7 on the validation set
map7_score = map_at_k(y_test.values, y_pred_prob)
print(f'MAP@7 Score: {map7_score:.4f}')

# Final predictions on the test set
y_test_prob = clf.predict_proba(test[feature_cols])

# Generate top 7 predictions for the test dataset
test_ids = test['ncodpers'].values
final_preds = []
for idx, pred in enumerate(y_test_prob):
    top_k = np.argsort(pred)[::-1][:7]
    final_preds.append(" ".join(targetcols[1:][top_k]))  # Map the indices to product names

# Prepare the submission
submission = pd.DataFrame({'ncodpers': test_ids, 'added_products': final_preds})
submission.to_csv('submission.csv', index=False)
print('Submission saved!')